In [22]:
ASSETS_DIR = "/content/drive/MyDrive/Colab Notebooks/assets"

In [1]:
from __future__ import annotations

import numpy as np
import pandas as pd

from dataclasses import dataclass, field

In [7]:
@dataclass
class Node:
  indexes: list[int] = field(default_factory=list)
  childs: list[Node] = field(default_factory=list)
  branch: str = ""
  key: str = ""

  def __show(self, indexes, branch, childs, key, r=0, c=0, x=0):
    print(c * '\t', indexes, end=" ")
    print("#" + str(branch) if branch else "", end=" ")
    print("@" + str(key) if key else "", end="\n")

    if len(childs) == 0:
      return
    else:
      c += 1

    for i in childs:
      r += 1
      x = c * r + 1
      self.__show(i.indexes, i.branch, i.childs, i.key, r, c, x)

  @property
  def show(self):
    self.__show(self.indexes, self.branch, self.childs, self.key)

In [23]:
def entropy(s):
    p = -s.value_counts() \
          .div(s.count()) \
          .agg(lambda x: x*np.log2(x)) \
          .sum()

    return p

In [2]:
def exclude_from(df, exclude_list):
  if len(exclude_list) > 0:
    return df.loc[:, ~df.columns.isin(exclude_list)]
  return df

---
# ID3

In [24]:
def gain(s, col, target):
    p_target = entropy(s[target])
    keys = s[col].drop_duplicates()
    entropies = keys.apply(lambda _: (_,
                                      entropy(s[target][s[col][s[col] == _].index])))
    ratios = s[col].value_counts().div(len(s))
    res = np.subtract(p_target,
                      entropies.apply(lambda _: ratios.loc[_[0]] * _[1])
                               .sum())

    return col, res

In [25]:
def id3(s, *, df=None, target, exclude, verbose=False):
    node = Node()

    # stopping case
    if df is not None and len(df) == 0:
        return

    # store first dataframe
    if df is None:
        df = s.copy()

    node.indexes = list(s.index)

    # remove excluded columns from dataframe
    s = exclude_from(s, exclude)
    #if len(exclude) > 0:
    #    s = s.loc[:, ~s.columns.isin(exclude)]

    columns = s.columns.drop(labels=[target])

    gains = columns.map(lambda x: gain(s, x, target)) \
                   .to_frame(index=False) \
                   .set_index(0)

    if verbose:
      print("\nSET:\n\n", df.loc[s.index])
      print("\nGAINS: \n", gains)

    if gains.empty or len(df[target].value_counts()) == 1:
        if verbose:
          print("\n\nLEAF<<<\n\n")
        node.branch = df[target].values[0]
        return node

    next_branch = gains.idxmax().values[0]
    node.branch = next_branch

    if verbose:
      print("\nNEXT BRANCH: ", next_branch)

    next_S = df[next_branch].drop_duplicates() \
                            .apply(lambda _: df.loc[df[next_branch][df[next_branch] == _] \
                                               .index])

    for k, v in enumerate(next_S):
      if verbose:
        print(f"epoch @{k}")
      #print(v[next_branch].values[0])
      n = id3(v, df=df.loc[v.index], target=target, exclude=[*exclude, next_branch])
      n.key = v[next_branch].values[0]
      node.childs.append(n)

    return node


In [26]:
df = pd.read_csv(ASSETS_DIR + "/weather.csv")
target = "play" #@param {type: "string"}
exclude = ["id"] #@param

# 1st epoch
tree = id3(df, exclude=exclude, target=target)
tree.show

 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] #outlook 
	 [0, 1, 7, 8, 10] #humidity @sunny
		 [0, 1, 7] #no @high
		 [8, 10] #yes @normal
	 [2, 6, 11, 12] #yes @overcast
	 [3, 4, 5, 9, 13] #windy @rainy
		 [3, 4, 9] #yes 
		 [5, 13] #no @True


---

In [27]:
df = pd.read_csv(ASSETS_DIR + "/job.csv")
target = "risk" #@param {type: "string"}
exclude = ["customer"] #@param

# 1st epoch
tree = id3(df, exclude=exclude, target=target, verbose=False)
tree.show

 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] #debt 
	 [0, 1, 2] #Bad @High
	 [3, 4, 5, 6, 7, 8, 9] #revenue @Low
		 [3, 4, 7, 8] #status @Low
			 [3, 7] #Good @Employee
			 [4, 8] #Bad @Employeer
		 [5, 6, 9] #Good @High


---

In [28]:
df = pd.read_csv(ASSETS_DIR + "/tennis.csv")
target = "play" #@param {type: "string"}
exclude = [] #@param

# 1st epoch
tree = id3(df, exclude=exclude, target=target)
tree.show

 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] #outlook 
	 [0, 1, 7, 8, 10] #humidity @sunny
		 [0, 1, 7] #no @high
		 [8, 10] #yes @normal
	 [2, 6, 11, 12] #yes @overcast
	 [3, 4, 5, 9, 13] #windy @rainy
		 [3, 4, 9] #yes @weak
		 [5, 13] #no @strong


---

In [ ]:
df = pd.read_csv(ASSETS_DIR + "/monks.csv")
target = "has_tie" #@param {type: "string"}
exclude = [] #@param

# 1st epoch
tree = id3(df, exclude=exclude, target=target)
tree.show

---

In [ ]:
df = pd.read_csv(ASSETS_DIR + "/optical.csv")
target = "label" #@param {type: "string"}
exclude = [] #@param

# 1st epoch
tree = id3(df, exclude=exclude, target=target)
tree.show

---
# C4.5

In [29]:
data = {
    "A1": ['a', 'a', 'a', 'a', 'a', 'b', 'b', 'b', 'b', 'c', 'c', 'c', 'c', 'c'],
    "A2": [70, 90, 85, 95, 70, 90, 78, 65, 75, 80, 70, 80, 70, 96],
    "A3": [True, True, False, False, False, True, False, True, False, True, True, False, False, False],
    "class": ["class1", "class2", "class2", "class2", "class1", "class1","class1","class1","class1", "class2", "class2", "class1", "class1","class1",]
}

df = pd.DataFrame(data)
threshold = df['A2'].drop_duplicates() \
                    .sort_values() \
                    .median()
df['A2'] = df['A2'].apply(lambda x: 'lte' if x <= threshold else 'gt')
df[:3]

,A1,A2,A3,class
0,a,lte,True,class1
1,a,gt,True,class2
2,a,gt,False,class2


In [30]:
target = "class" #@param {type: "string"}
exclude = [] #@param

# 1st epoch
tree = id3(df, exclude=exclude, target=target, verbose=False)
tree.show

 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13] #A1 
	 [0, 1, 2, 3, 4] #A2 @a
		 [0, 4] #class1 @lte
		 [1, 2, 3] #class2 @gt
	 [5, 6, 7, 8] #class1 @b
	 [9, 10, 11, 12, 13] #A3 @c
		 [9, 10] #class2 @True
		 [11, 12, 13] #class1 


---
# TWOING

In [3]:
a = {
    "salary": ["N", "H", "L", "H", "L", "H", "L", "H", "L", "H", "L"],
     "exp": ["M", "N", "N", "M", "M", "G", "G", "M", "M", "G", "G"],
     "major": ["expert", "expert", "manager", "manager","manager","manager","manager", "expert", "expert","expert","expert"],
     "satisfied": ["Y", "Y", "Y", "Y", "Y", "Y", "Y", "N", "N","N","N"]
}

target = "satisfied"
exclude = []

df = pd.DataFrame(a)
df[:3]

,salary,exp,major,satisfied
0,N,M,expert,Y
1,H,N,expert,Y
2,L,N,manager,Y


In [4]:
import itertools as it

def get_permutations(df):
  permutations = {}
  for _ in df.iloc[:, :-1]:
      p = pd.DataFrame(it.permutations(df[_].unique())) \
            .drop_duplicates(subset=[0]) \
            .reset_index(drop=True)
      permutations[_] = p

  return permutations

In [20]:
import collections

def twoing(df, *, target, exclude=[]):
  def solve(p, k):
    def P(r):
      l = df[df[k].isin(r)]
      ll = len(l)
      pl = np.divide(ll, len(df))
      pr = l.groupby(target) \
            .count()[k] \
            .apply(lambda x: np.divide(x, ll))
  
      #pr.index = pr.index.set_names([''])
      #pr = pd.DataFrame(pr)
      #pr['p'] = pr[k].apply(lambda x: np.divide(x, ll))
  
      return ll, pl, pr

    epochs = []
    for i in p:
      L, R = p.iloc[i, :1], p.iloc[i, 1:]
      bl, lpl, lpr = P(L)
      br, rpl, rpr = P(R)
  
      #print(f"P-LEFT={lpl:.2f} \t P-RIGHT={rpl:.2f}")
  
      val = (2 * lpl * rpl * lpr.sub(rpr, fill_value=0.0)
                                .abs() 
                                .sum())
      
      epochs.append(val)
      #print(f"{val=:.2f}")
      #print('\n\n' + 32*'-')

    return epochs

  # stopping case
  if df is not None and len(df) == 0:
      return

  node = Node()
  node.indexes = list(df.index)

  excluded_df = exclude_from(df, exclude)
  #print(excluded_df)

  permutations = get_permutations(excluded_df)
  epochs = collections.defaultdict(list)
  for k, v in permutations.items():
    e = solve(v, k)
    epochs[k] = e

  if len(epochs) == 0 or len(df[target].value_counts()) == 1:
      node.branch = df[target].values[0]
      return node
  
  next_branch = max(epochs, key=epochs.get)
  node.branch = next_branch

  #print(next_branch)
  #print(permutations[next_branch][0])
  #input(">>>")
  
  for i in permutations[next_branch][0]:
    n = twoing(df[df[next_branch] == i], target=target, exclude=[*exclude, next_branch])
    node.childs.append(n)

  return node

In [21]:
tree = twoing(df, target="satisfied")
tree.show

 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10] #major 
	 [0, 1, 7, 8, 9, 10] #salary 
		 [0] #Y 
		 [1, 7, 9] #exp 
			 [1] #Y 
			 [7] #N 
			 [9] #N 
		 [8, 10] #N 
	 [2, 3, 4, 5, 6] #Y 
